사용 model : xgb, lgbm, cat, RF, inceptionV3  
  
table데이터만을 사용한 모델과 이미지만을 사용한 모델을 각각 만든 후 예측  
예측한 결과(확률)를 비율로 섞어 최종 예측을 만든 후 제출  
table 데이터 : 80%(xgb, lgbm, cat, rf 각각 20%씩) 이미지 데이터 : 20%

In [ ]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import cv2
import albumentations as A
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

GPU 사용

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

Data받기(수술연월일 = parse_date로 받기)

In [ ]:
path = 'D:/Dataset/Competitons/cancer/'
train = pd.read_csv(path + 'train.csv', parse_dates=["수술연월일"]).copy()
test = pd.read_csv(path + 'test.csv', parse_dates=["수술연월일"]).copy()
clinical_info = pd.read_excel(path + 'clinical_info.xlsx').copy()

In [ ]:
train

비어있는 값 확인

In [ ]:
train.isna().sum()

column 이름 확인

In [ ]:
train.columns.unique()

'수술연월일'을 year, month, day, quarter로 쪼개서 새 columns로 추가

In [ ]:
def date_split(df):

    df['year'] = df['수술연월일'].dt.year
    df['month'] = df['수술연월일'].dt.month
    df['day'] = df['수술연월일'].dt.day
    df['quarter'] = df['수술연월일'].dt.quarter

    return df
    
train = date_split(train)
test = date_split(test)
train

비어있는 값 채우기

In [ ]:
train['암의 장경'] = train['암의 장경'].fillna(train['암의 장경'].mean())
train = train.fillna(0)

test['암의 장경'] = test['암의 장경'].fillna(train['암의 장경'].mean())
test = test.fillna(0)

label값 복사

In [ ]:
label = train['N_category'].values.copy()

ID, mask_path 제거 후 각각 train_table, test_table로 저장  
현재의 모델은 table데이터들만 사용하는 모델이지만, img_path는 나중에 따로 뺀 후 제거 예정

In [ ]:
ignore_cols_train = ['ID', 'mask_path', '수술연월일', 'N_category']
ignore_cols_test = ['ID', 'img_path', '수술연월일']

train_table = train.drop(ignore_cols_train, axis = 1).copy()
test_table = test.drop(ignore_cols_test, axis = 1).copy()
train_table

train_test_split으로 val데이터 생성(성능 check위해)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train_table, label, random_state = 42)

In [ ]:
X_train

train에 있는 img_path만 따로 저장(각각 X_train_img 와 X_val_img)  
현재 dataset은 코드저장되어있는 부분과 다른 곳에 위치하므로, path수정후 저장

In [ ]:
X_train_img = X_train['img_path'].values.copy()
X_train_img = [path + j[2:] for j in X_train_img]
X_train_img = np.reshape(X_train_img, (len(X_train_img)))

X_val_img = X_val['img_path'].values.copy()
X_val_img = [path + j[2:] for j in X_val_img]
X_val_img = np.reshape(X_val_img, (len(X_val_img)))

img_path 따로 뺐으니, X_train, X_val에서 img_path 제거

In [ ]:
ignore_img = ['img_path']

X_train = X_train.drop(ignore_img, axis = 1)
X_val = X_val.drop(ignore_img, axis = 1)

XGBClassifier  
xgboost가 없는 경우 : !pip install xgboost

In [ ]:
from xgboost import XGBRFClassifier

xgb = XGBRFClassifier(random_state = 42)
xgb.fit(X_train, y_train, eval_set = [(X_val, y_val)])

test_table 예측 확인

In [ ]:
xgb.predict(test_table)

test_table 예측 한 확률을 각각 출력후 저장  
예측 확률로 한 이유 ; 이후 각 예측 확률을 더해 최종 예측을 만들기 위해

In [ ]:
xgb_pred = xgb.predict_proba(test_table)

CatBoostClassifier  
catboost가 없는 경우 : !pip install catboost

In [ ]:
from catboost import CatBoostClassifier

cat = CatBoostClassifier(random_state = 42, verbose = 0)
cat.fit(X_train, y_train, eval_set = [(X_val, y_val)])

test_table cat으로 예측 확인

In [ ]:
cat.predict(test_table)

예측 확률 저장

In [ ]:
cat_pred = cat.predict_proba(test_table)

ExtraTreesClassifier

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

ex_clf = ExtraTreesClassifier(random_state = 42)
ex_clf.fit(X_train, y_train)

In [ ]:
ex_clf.predict(test_table)

In [ ]:
ex_pred = ex_clf.predict_proba(test_table)

RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state = 42)
rf.fit(X_train, y_train)

In [ ]:
rf.predict(test_table)

In [ ]:
rf_pred = rf.predict_proba(test_table)

LGBMClassifier  
lightgbm이 없는 경우 : !pip install lightgbm

In [ ]:
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier(random_state = 42)
lgbm.fit(X_train, y_train)

In [ ]:
lgbm.predict(test_table)

In [ ]:
lgbm_pred = lgbm.predict_proba(test_table)

f1_score확인  
성능 확인 방법 : model에 넣지 않은 val을 넣고 성능 확인

In [ ]:
from sklearn.metrics import f1_score
def scoring(y, pred):
    score = f1_score(y_true = y, y_pred = pred)
    print(score)

 score찍어서 확인

In [ ]:
model = [xgb, cat, rf, ex_clf, lgbm]
for i in model:
    print(str(i))
    print(i.score(X_val, y_val))

f1_score 점수 확인

In [ ]:
for i in model:
    print(str(i))
    print(scoring(y_val, i.predict(X_val)))

Extra는 RF와 비슷하므로 둘 중 낮은 값인 Extra는 사용 X

Image모델(InceptionV3 사용)

test의 img_path 역시 dataset이 위치한 곳으로 수정 후 따로 저장

In [ ]:
test_img = test['img_path'].values.copy()
test_img = [path + j[2:] for j in test_img]
test_img = np.reshape(test_img, (len(test_img)))

잘 수정되었는지 확인

In [ ]:
X_train_img[0]

img_size 지정 img_resize

In [ ]:
img_size = 512
img_channel = 3
batch = 8

resize_crop = A.Compose([
    A.Resize(img_size, img_size, interpolation = cv2.INTER_AREA)
])

CustomSequence제작  
img dataset을 사용한 이유 : 데이터 양이 너무 커서 컴퓨터가 버티질 못해 사용....ㅠ  
dataset을 사용하니 메모리가 터지지는 않으나, 너무 느려짐.....ㅠㅜㅠㅜㅠ

In [ ]:
from tensorflow.keras.utils import Sequence
from sklearn.utils import shuffle
from tensorflow.keras.applications.xception import preprocess_input
import math

class CustomSequence(Sequence):
    def __init__(self, filenames, target, batch_size, aug=None, shuffle=False):
        # image의 절대경로들
        self.filenames = filenames
        self.target = target
        self.batch_size = batch_size
        # albumentation 객체
        self.aug = aug
        self.shuffle = shuffle
        
        # 훈련 데이터의 경우
        if self.shuffle:
            self.on_epoch_end()

    def __len__(self):
        # 총 step의 갯수
        return math.ceil(len(self.filenames) / self.batch_size)

    def __getitem__(self, index):
        # 현재 인덱스를 기준으로 batch_size만큼 데이터를 가져옴
        meta_data = self.filenames[index*self.batch_size:(index+1)*self.batch_size]
        # 훈련, 검증 데이터세트인 경우
        if self.target is not None:
            label_batch = self.target[index*self.batch_size:(index+1)*self.batch_size]

        # 불러온 meta_data를 np.array로 저장할 빈 공간을 생성
        image_batch = np.zeros((meta_data.shape[0], img_size, img_size, img_channel))
        for i in range(meta_data.shape[0]):
            # cv2는 이미지를 BGR로 불러오기 때문이 RGB로 바꾸어줌
            image = cv2.cvtColor(cv2.imread(meta_data[i]), cv2.COLOR_BGR2RGB)
            # augmentation이 있으면 적용
            if self.aug is not None:
                image = self.aug(image=image)['image']

            # 이미지 값을 self.pre_func 함수로 스케일링
            image = image * 1./255

            # 빈 이미지 배치에 최종 이미지를 등록
            image_batch[i] = image
            
        if self.target is not None:
            return image_batch, label_batch
        else:
            return image_batch
        
    def on_epoch_end(self):
        # 파일과 라벨을 같이 섞어 주어야한다.
        if self.shuffle is True:
            self.image_filenames, self.target = shuffle(self.filenames, self.target)
        else:
            pass

In [ ]:
train_dataset = CustomSequence(X_train_img, y_train, batch, resize_crop)
val_dataset = CustomSequence(X_val_img, y_val, batch, resize_crop)

In [ ]:
len(train_dataset)

dataset이 잘 되었는지 확인용

In [ ]:
def show_first_data(dataset, image_verbose=False):
    first_data = next(iter(dataset))
    images = first_data[0]
    labels = first_data[1]
    print(images.shape, labels.shape)
    print(images[0])
    if image_verbose:
        fig, axs = plt.subplots(figsize=(20, 8), nrows=1, ncols=4)
        for i in range(4):
            axs[i].imshow(np.array(images[i], dtype='int32'))

In [ ]:
show_first_data(train_dataset)

CNN 모델 생성(inceptionV3)

In [ ]:
def make_model():
    base_model = keras.applications.InceptionV3(include_top = False, weights = 'imagenet', input_shape = (img_size, img_size, 3), pooling = None)
    x = base_model.output
    x = keras.layers.GlobalAveragePooling2D()(x)
    x = keras.layers.Dense(512, kernel_initializer = 'lecun_normal', activation = 'selu')(x)
    x = keras.layers.Dense(256, kernel_initializer = 'lecun_normal', activation = 'selu')(x)
    x = keras.layers.Dense(128, kernel_initializer = 'lecun_normal', activation = 'selu')(x)
    x = keras.layers.Dense(64, kernel_initializer = 'lecun_normal', activation = 'selu')(x)
    predictions = keras.layers.Dense(1, activation = 'sigmoid')(x)

    model = keras.models.Model(inputs=base_model.input, outputs=predictions)    
    return model

In [ ]:
img_model = make_model()

callback함수(ReduceLROnPlateau, EarlyStopping, Checkpoint)  
  
val로 성능 확인하면서 3번 이상 성능 감소 안하면 learning late감소, 10번 val 성능이 감소 안하면 정지,  
가장 좋은 모델로 Callback)  

In [ ]:
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, mode = 'min')
ES = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
checkpoint_cb = keras.callbacks.ModelCheckpoint('img_model.h5', save_best_only=True)

In [ ]:
img_model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])

img_model.fit(train_dataset, batch_size = batch,
              validation_data = val_dataset,
              epochs=100, callbacks=[reduce_lr, ES, checkpoint_cb],
              verbose = 1)

epoch가 더 늘어날 수도, 줄어들 수도 있음(제출 epoch는 16에서 정지 -> epoch = 6)

test_img도 dataset으로 제작

In [ ]:
test_dataset = CustomSequence(test_img, None, batch, resize_crop)

In [ ]:
test_dataset

CNN에서 predict를 하면, 0에서 1 사이의 값이 나온다.(확률)  
이는 1로 될 확률과 같으므로, 0일 때의 확률은 1 - predict이고, 1일 때의 확률은 predict로 하는 array를 생성한다.

In [ ]:
def img_predict_proba(model, test_dataset):
    img_pred = model.predict(test_dataset)
    pred_test = np.zeros((len(img_pred), 2))

    for i in range(0, len(img_pred)):
        pred_test[i][0] = (1 - img_pred[i])
        pred_test[i][1] = img_pred[i]

    return pred_test

In [ ]:
img_pred = img_predict_proba(img_model, test_dataset)

val_dataset으로 f1_score 확인

In [ ]:
scoring(y_val, np.argmax(img_predict_proba(img_model, val_dataset), axis = 1))

xgb, rf, cat, lgbm과 img 확률들을 더해 최종 예측 생성(각각 0.2씩)

In [ ]:
pred = (img_pred + xgb_pred + rf_pred + cat_pred + lgbm_pred) / 5
pred

np.argmax를 이용해 확인

In [ ]:
np.argmax(pred, axis = 1)

submission csv생성

In [ ]:
output = pd.DataFrame({'ID' : test['ID'], 'N_category':np.argmax(pred, axis = 1)})
output.to_csv('final_school.csv', index=False)
print("Your submission was successfully saved!")